# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core import Workspace, Experiment
from azureml.data.dataset_factory import TabularDatasetFactory
from sklearn.model_selection import train_test_split
from azureml.core import Dataset
from azureml.train.automl import AutoMLConfig

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'capstone-azure-ml'

experiment=Experiment(ws, experiment_name)
project_folder = './pipeline-project'

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code RWU5J5XTT to authenticate.
You have logged in. Now let us find all the subscriptions to which you have access...
Interactive authentication successfully completed.


In [3]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster
amlcompute_cluster_name = "auto-ml"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 3)
# For a more detailed view of current AmlCompute status, use get_status().




Found existing cluster, use it.
Succeeded...................................
AmlCompute wait for completion finished

Wait timeout has been reached
Current provisioning state of AmlCompute is "Succeeded" and current node count is "0"


In [4]:



#Dataset

found = False
key = "capstoneDataset"
description_text = "Heart DataSet for Udacity Capstone"

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 
        print("found!")

if not found:
        # Create AML Dataset and register it into Workspace
        print("not found!")
        example_data = 'heart.csv'
        dataset = Dataset.Tabular.from_delimited_files(example_data)        
        #Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)


df = dataset.to_pandas_dataframe()
##df.describe()




found!


In [5]:
import pandas as pd
import numpy as np

##df = pd.read_csv('C:/Dataset.csv')
df['split'] = np.random.randn(df.shape[0], 1)

msk = np.random.rand(len(df)) <= 0.7

train = df[msk]
test = df[~msk]

train.describe()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target,split
count,221.000000,221.000000,221.000000,221.000000,221.000000,221.000000,221.000000,221.000000,221.000000,221.000000,221.000000,221.000000,221.000000,221.000000,221.000000
mean,54.221719,0.687783,0.972851,131.366516,246.900452,0.176471,0.556561,149.610860,0.316742,1.031222,1.393665,0.701357,2.316742,0.533937,0.006426
std,9.142343,0.464450,1.030968,18.478650,54.335900,0.382085,0.524591,22.574778,0.466262,1.190253,0.627952,0.986936,0.631837,0.499979,1.015397
min,29.000000,0.000000,0.000000,94.000000,126.000000,0.000000,0.000000,71.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-2.401413
25%,47.000000,0.000000,0.000000,120.000000,211.000000,0.000000,0.000000,136.000000,0.000000,0.000000,1.000000,0.000000,2.000000,0.000000,-0.686353
50%,55.000000,1.000000,1.000000,130.000000,241.000000,0.000000,1.000000,153.000000,0.000000,0.600000,1.000000,0.000000,2.000000,1.000000,-0.043916
75%,61.000000,1.000000,2.000000,140.000000,277.000000,0.000000,1.000000,164.000000,1.000000,1.600000,2.000000,1.000000,3.000000,1.000000,0.651580
max,77.000000,1.000000,3.000000,200.000000,564.000000,1.000000,2.000000,202.000000,1.000000,6.200000,2.000000,4.000000,3.000000,1.000000,3.100448


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [6]:
pd.DataFrame(train).to_csv("data/train_data.csv")
ds = ws.get_default_datastore()
ds.upload(src_dir='./data', target_path='HeartDataSet', overwrite=True, show_progress=True)
# Upload the training data as a tabular dataset for access during training on remote compute
train_data = TabularDatasetFactory.from_delimited_files(path=ds.path('HeartDataSet/train_data.csv'))

Uploading an estimated of 1 files
Uploading ./data/train_data.csv
Uploaded ./data/train_data.csv, 1 files out of an estimated total of 1
Uploaded 1 files


In [7]:
# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes" : 25,
    "max_concurrent_iterations": 4,
    "n_cross_validations": 5,
    "primary_metric": 'accuracy',
}
automl_config = AutoMLConfig(task = 'classification',
                             debug_log = 'automl_errors.log',
                             compute_target=compute_target,
                             training_data = train_data,
                             label_column_name = 'target',
                             **automl_settings
                            )

print("done.")

done.


In [8]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config, show_output= True)

Running on remote.
No run_configuration provided, running on auto-ml with default configuration
Running on remote compute: auto-ml
Parent Run ID: AutoML_b6c7b20b-0982-41d1-98ce-4f43694219ec

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [10]:
from azureml.widgets import RunDetails
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [13]:
best_run, fitted_model = remote_run.get_output()

In [14]:
best_run

Experiment,Id,Type,Status,Details Page,Docs Page
capstone-azure-ml,AutoML_b6c7b20b-0982-41d1-98ce-4f43694219ec_34,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [15]:
best_run.get_metrics()

{'average_precision_score_weighted': 1.0,
 'AUC_weighted': 1.0,
 'AUC_micro': 1.0,
 'average_precision_score_macro': 1.0,
 'f1_score_weighted': 1.0,
 'f1_score_micro': 1.0,
 'f1_score_macro': 1.0,
 'precision_score_macro': 1.0,
 'balanced_accuracy': 1.0,
 'norm_macro_recall': 1.0,
 'recall_score_micro': 1.0,
 'precision_score_weighted': 1.0,
 'precision_score_micro': 1.0,
 'AUC_macro': 1.0,
 'recall_score_macro': 1.0,
 'matthews_correlation': 1.0,
 'average_precision_score_micro': 1.0,
 'accuracy': 1.0,
 'weighted_accuracy': 1.0,
 'recall_score_weighted': 1.0,
 'log_loss': 0.08149825423180843,
 'accuracy_table': 'aml://artifactId/ExperimentRun/dcid.AutoML_b6c7b20b-0982-41d1-98ce-4f43694219ec_34/accuracy_table',
 'confusion_matrix': 'aml://artifactId/ExperimentRun/dcid.AutoML_b6c7b20b-0982-41d1-98ce-4f43694219ec_34/confusion_matrix'}

In [16]:
#TODO: Save the best model


## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [18]:
description = 'My AutoML Model'
model = remote_run.register_model(description = description)

print(remote_run.model_id)

AutoMLb6c7b20b034


TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service